In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

In [2]:
service = ChromeService(executable_path=ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:00<00:00, 23.1MB/s]


In [3]:
driver = webdriver.Chrome(service=service)
url ='https://www.google.com/maps/place/York+Hospital/@53.9634251,-1.0954656,14z/data=!4m18!1m9!3m8!1s0x487931a5d58341b1:0x97daca52f7800d35!2sYork+Minster!8m2!3d53.9623292!4d-1.0819205!9m1!1b1!16zL20vMGptczE!3m7!1s0x487931a1db7f2029:0xbb7ef99dc5f7ebc4!8m2!3d53.9699817!4d-1.083555!9m1!1b1!16s%2Fm%2F0gvrhzd'
driver.get(url)
driver.find_element(By.CSS_SELECTOR, "[aria-label='Godkänn alla']").click()

In [4]:
driver.find_element(By.CLASS_NAME, "HQzyZ").click()
recent = driver.find_elements(By.XPATH, '//div[@role=\'menuitemradio\']')[1]
recent.click()
time.sleep(5)

In [5]:

for i in range(0,30):
        scrollable_div = driver.find_elements(By.CLASS_NAME,'DU9Pgb') 
        div_i = scrollable_div[len(scrollable_div)-1]
        driver.execute_script('arguments[0].scrollIntoView()', div_i)
        time.sleep(1)


In [6]:
num=len(WebDriverWait(driver,30).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "[aria-label^='Visa mer']"))))
for i in range(num):
    button=driver.find_element(By.CSS_SELECTOR,  "[aria-label^='Visa mer']")
    driver.implicitly_wait(10)
    ActionChains(driver).move_to_element(button).click(button).perform()

In [7]:
response = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
star_list =[]
review_contents_list = []

for i in response.find_all  ("div",{"class": "DU9Pgb"}):
    star = i.find("span",{"role":"img"})
    star_num=0
    for image in i.find_all('img',{"class":"hCCjke vzX5Ic"}):
        star_num +=1
    star_list.append(star_num)

    if i.find_next_sibling("div").find("div", {"class":"MyEned"}):
        review_contents = i.find_next_sibling("div").find("span", {"class":"wiI7pd"}).getText()
    else:
        review_contents=""
    review_contents_list.append(review_contents)
        

zipped = list (zip(star_list, review_contents_list))
hospital_review = pd.DataFrame(zipped, columns=['star', 'review_content'])

In [10]:
hospital_review = hospital_review[hospital_review["review_content"]!=""]

In [12]:
hospital_review.to_csv("hospital_review.csv")

Reference 
https://towardsdatascience.com/scraping-google-maps-reviews-in-python-2b153c655fc2
https://medium.com/@isguzarsezgin/scraping-google-reviews-with-selenium-python-23135ffcc331